In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import scipy.signal as sig

wwt tutorial

In [ ]:
% Wavelets - a practical walk-though
% This script shows an example wavelet analysis with 1-fold connectivity matrix, relative to sensor Oz, while calculating all necessary information for publication in a peer-reviewed paper. 
 % first load exampe data, make sure the data are in the current directory,
 % or in the path
load('bop_106.fl40h1.E1.app1.mat') % loads our example data set (129 sensors, 2551 points, 59 trials)
size(outmat)
SampRate = 500; 
% Experimental design and trial structure for the example data


% Now, calculate the frequency steps for the wavelet, the resolution/stepwidth will need to be mentioned in your manuscript
freqRes = 1000/(2551*2) % = 1000/5102 ms (2551 sample points sampled at 500 Hz)
% this is the best possible  frequency resolution (the most steps you can have). 
freqs_allpossible = 0:freqRes:SampRate/2; % 1276 possible frequencies you can look at but that is too much, and unnecessarily dense 

freqs_wanted = freqs_allpossible(20: 4: 200); % look at frequencies from 3.72 to 39 Hz 
%this applies to everything that is based on a frequency transformation, not just for wavelets. You can only go worse than your native (best)

% frequency resolution i.e. 1/(length in secons) Hz,  but you cannot make it better. 

f0start = 20 %This is *NOT* in Hz! this is wavelet units, see lecture slides and above
f0end = 200; % stop at 
fdelta = 4; % calculate only every 4th wavelet of the ones you could do
refelec = 62; % This is Pz
outname = 'test' % this is also known as the worst filename :-) 

[WaPower, PLI, PLIdiff] = wavelet_app_mat(outmat, SampRate, f0start, f0end, fdelta, refelec, outname);

% phase locking, inter-site phase locking
% sensor layout, to select seed electrodes (refelec) for inter-site phase locking



timeaxis = -3600:2:1500; 

PowerBsl = bslcorrWAMat_div(WaPower, [40:140]);
 % select a baseline away from beginning and end of 
                                                 % the pre-stimulus period
                                                 % % try different ones and
                                                 % see what happens below
                                                 % in figure 3
                                                 
 %length of baseline should be maximally sigT from beginning and also t0 - sigT 
 %to avoid onset artefacts and anything that happens after stimulus onset
 %you should have at least a cycle of your lowest frequency
 %sigmaT=1/(2*pi*sigmaF)

PowerdB = 10*log10(PowerBsl); 


% And now, a bunch of figures:
figure, contourf(timeaxis(50:2500), freqs_wanted, squeeze(WaPower(75, 50:2500, :))'), colorbar
title('Raw power with no baseline correction')

figure, contourf(timeaxis(50:2500), freqs_wanted, squeeze(PowerBsl(75, 50:2500, :))'), colorbar
title('baseline-corrected power (simple divide)')

figure, contourf(timeaxis(50:2500), freqs_wanted, squeeze(PowerdB(75, 50:2500, :))'), colorbar
title('baseline-corrected power (decibels)')

% no baseline correction for Phase-locking - do you know why ? 


figure, contourf(timeaxis(50:2500), freqs_wanted, squeeze(PLI(75, 50:2500, :))'), colorbar
title('phase locking across trials at sensor 75')
 
figure, contourf(timeaxis(50:2500), freqs_wanted, squeeze(PLIdiff(75, 50:2500, :))'), colorbar
title('phase locking between sensor 75 and 62 (Oz and Pz) across sensors and trials ')

% baseline corrected power (power change)


% edit gener_wav  % change m and see what happens when you do it again...
% here you can change the m parameter; setting a lower m will give more
% resolution in higher frequencies. 

% Measure the time and frequency resolution with an impulse response. This is an alternative to calculating the time and frequency resolution as discussed in the slides (day 2) 

pulse = zeros(1,701);

pulse(351)=1;

figure, plot(pulse)

[WaPowerP] = wavelet_app_mat(pulse, 500, 13, 60, 1);

figure, contourf(timeaxis(50:650), freqs_wanted(1:40), squeeze(WaPowerP(1, 50:650, 1:40))', 20), colorbar



wwt function

In [ ]:
% wavelet on 3-d matrix
function [WaPower, PLI, PLIdiff] = wavelet_app_mat(mat, SampRate, f0start, f0end, fdelta, refelec, outname); 

if nargin < 7, 
   save_flag = 0; 
 else,   save_flag = 1; 
end


if nargin < 6, 
    refelec = []; 
    PLIdiff = []; 
end

PLIsum = []; 
 PLIdiff = []; 

 
data = mat;

  NPointsNew = size(data,2);   
  NTrials = size(data,3);   

  
  % compute wavelets and their parameters 
  wavelet = gener_wav(NPointsNew, fdelta, f0start, f0end);
 
  disp('size of waveletMatrix')
  disp(size(wavelet))
  disp (' frequency step for delta_f0 = 1 is ')
  disp(SampRate/NPointsNew)
 
    
   % create 3d matrix objects for wavelet
   % channels * time * frequencies
   
    waveletMat3d = repmat(wavelet, [1 1 size(data,1)]); 
    waveletMat3d = permute(waveletMat3d, [3, 2, 1]); 
    
    % loop over trials
    
    disp(['trial index of '])
    disp(NTrials)
    
    for trialindex = 1:NTrials;  
        
        Data = data(:,:,trialindex);
        
        fprintf('.')
         
        if trialindex/10 == round(trialindex/10), disp(trialindex), end
    
        Data = bslcorr(Data, 1:200); 
        
       size(Data);

        % window data with cosine square window
        window = cosinwin(50, size(Data,2), size(Data,1)); 
        
        Data = Data .* window; 
        
        data_pad3d = repmat(Data, [1 1 size(wavelet,1)]); 
    
        % transform data  to the frequency domain

        data_trans = fft(data_pad3d, NPointsNew, 2);

        thetaMATLABretrans = []; 

        ProdMat= waveletMat3d .*(data_trans);

        thetaMATLABretrans = ifft(ProdMat, NPointsNew, 2);
        
        % standardize instantaneous phase
        stdcomplexphasemat = thetaMATLABretrans ./ abs(thetaMATLABretrans);
        
        % if a reference sensor is given, calculate inter-site PLI for that
        % sensor
        if ~isempty (refelec)
            PLIdiffmat = stdcomplexphasemat- (repmat(stdcomplexphasemat(refelec,:,:), [size(mat,1), 1, 1]));       
            PLIdiffmat(refelec,:,:) = ones(size( PLIdiffmat(refelec,:,:))); 
            %standardize the difference
            PLIdiffmat = PLIdiffmat./(abs(PLIdiffmat)); 
        end
        
              
        if trialindex == 1
            WaPowerSum = abs(thetaMATLABretrans).* 10; 
            PLIsum = stdcomplexphasemat; 
                if ~isempty (refelec)
            PLIdiffsum = PLIdiffmat; 
                end
        else
        WaPowerSum = WaPowerSum + abs(thetaMATLABretrans).* 10; 
        PLIsum = PLIsum+ stdcomplexphasemat;
           if ~isempty (refelec)
               PLIdiffsum = PLIdiffsum+ PLIdiffmat; 
           end
        end

    end % loop over trials
    

    %final calculations: avg (and abs for PLI)
    
    WaPower = WaPowerSum./NTrials; 
   
    PLI = abs(PLIsum./NTrials); 
    
       if ~isempty (refelec)
         PLIdiff = abs(PLIdiffsum./NTrials); 
       end

     
   if save_flag    
     eval(['save ' outname '.pow3.mat WaPower'])
     eval(['save ' outname '.pli3.mat PLI'])
       if ~isempty (refelec),  eval(['save ' outname '.ispl3_' num2str(refelec) '.mat PLIdiff']), end
   end
%       
%       fclose('all') 
% 
% 



